In [ ]:
from nbdev import *

In [ ]:
# default_exp text_norm

# Text Normalization
> Functions used for TTS Dataset Preparation

In [ ]:
#export
import re
from typing import Tuple
from razdel import tokenize

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

## Functions for Pipeline

In [ ]:
#export
def collapse_whitespace(text: str) -> str:
    "Replace multiple various whitespaces with a single space, strip leading and trailing spaces."
    
    return re.sub(r'[\s\ufeff\u200b\u2060]+', ' ', text).strip()

In [ ]:
test_eq(collapse_whitespace(
    chr(int("0xfeff", 16)) + # zero width no-break space
    chr(int("0x200b", 16)) + # zero width space
    chr(int("0x202f", 16)) + # narrow no-break space
    chr(int("0x2060", 16)) + # word joiner
    chr(int("0x3000", 16)) + # ideographic space
    chr(int("0xa0"  , 16)) + # no-break space
    "\t\n 1 2   3     4     5       \t\r\n"),
    "1 2 3 4 5")

In [ ]:
#export
def lowercase(text: str) -> str:
    "Convert `text` to lower case."
    
    return text.lower()

In [ ]:
test_eq(lowercase('ПрИвеТ, ЧуВАК!'), 
                  'привет, чувак!')

In [ ]:
#export
def check_no_numbers(text: str) -> list:
    "Return a list of digits, or empty list, if not found."
    
    return re.findall(r'(\d+)', text)

In [ ]:
test_eq(check_no_numbers('Цифры есть 1 12 13.4'), ['1', '12', '13', '4'])
test_eq(check_no_numbers('Цифр нет'), [])

In [ ]:
#export
_specials = [(re.compile(f'{x[0]}'), x[1]) for x in [
    (r'\(?\d\d[:.]\d\d\)?', ''),  # timestamps
    (r'!\.{1,}', '!'), # !. -> !
    (r'\?\.{1,}', '?'),# ?. -> ?
    (r'\/', ''),
    (r'[\*\_]', ''),
    (r'[\(\)]', '')
    ]]

In [ ]:
#export
def remove_specials(text: str, purge_digits: bool=None) -> str:
    "Replace predefined in `_specials` sequence of characters"
    
    for regex, replacement in _specials:
        text = re.sub(regex, replacement, text)
    if purge_digits:
        text = re.sub(r'\d', '', text)
    return text

In [ ]:
#export
def purge_dots(text, purgedots=False):
    "If `purgedots`, `...`|`…` will be purged. Else replaced with `.`"
    text = re.sub(r'\s(…)', ' ', text)
    replacement = '' if purgedots else '.'
    text = re.sub(r'…', replacement, text)
    text = re.sub(r'\.{3}', replacement, text)
    text = re.sub(r'\.{2}', '', text)   # pause .. removed
    return text

In [ ]:
test_eq(purge_dots("Word..."), 'Word.')
test_eq(purge_dots("Word…",), 'Word.')

test_eq(purge_dots("Word...", purgedots=True), 'Word')
test_eq(purge_dots("Word…",   purgedots=True), 'Word')

test_eq(purge_dots(" …Word",), ' Word')

test_eq(purge_dots("Word..",), 'Word')

test_eq(purge_dots('Многоточие... Многоточие… … …Многоточие'),
                        'Многоточие. Многоточие.  Многоточие')

In [ ]:
test_eq(remove_specials('Скобки у аббревиатур (вайфай) удаляем.'),'Скобки у аббревиатур вайфай удаляем.')

In [ ]:
test_eq(remove_specials('Метки времени 01:12 или 01.01, (01:12) или (01.01) удаляем.'),
                        'Метки времени  или ,  или  удаляем.')
test_eq(remove_specials('Ой!. Ага?. / Стоп.'), 'Ой! Ага?  Стоп.')
test_eq(remove_specials('*США* _Френсис_'), 'США Френсис')

In [ ]:
#export
_abbreviations = [(re.compile(f'\\b{x[0]}', re.IGNORECASE), x[1]) for x in [
  (r'т\.е\.', 'то есть'),
  (r'т\.к\.', 'так как'),
  (r'и т\.д\.', 'и так далее.'),
  (r'и т\.п\.', 'и тому подобное.')
]]

In [ ]:
#export
def expand_abbreviations(text: str) -> str:
    "`expand_abbreviations()` defined in `_abbreviations`"
    
    for regex, replacement in _abbreviations:
        text = re.sub(regex, replacement, text)
    return text

In [ ]:
test_eq(
    expand_abbreviations('Привет Джон, т.е. Иван. Т.к. русский. И т.д. И т.п.'),
                         'Привет Джон, то есть Иван. так как русский. и так далее. и тому подобное.') 

In [ ]:
#export
def unify_dash_hyphen(text: str) -> str:
    "Unify dash and hyphen symbols -- replace with emdash or hyphen, separate with space."
    
    text = re.sub('[\u2212\u2012\u2014]', '\u2013', text) # replace minus sign, figure dash, em dash with en dash
    text = re.sub('[\u2010\u2011]', '\u002d', text)  # hyphen, non-breaking hyphen
    text = re.sub('\s*?(\u2013)\s*?',' \g<1> ',text)
    return text

In [ ]:
test_eq(unify_dash_hyphen(
    chr(int("2212",16))+ # minus sign
    chr(int("2012",16))+ # figure dash
    chr(int("2010",16))+ # hyphen
    chr(int("2011",16))),# non-breaking hyphen
    (" "+chr(int("2013",16))+" ")*2+chr(int("2d",16))*2)

In [ ]:
test_eq(unify_dash_hyphen('Я '+chr(int("2013",16))+ 'Джейми Кейлер'),'Я – Джейми Кейлер')
test_eq(unify_dash_hyphen('Я' +chr(int("2013",16))+ 'Джейми Кейлер'),'Я – Джейми Кейлер')
test_eq(collapse_whitespace(unify_dash_hyphen('Я' +chr(int("2013",16))+' Джейми Кейлер')),'Я – Джейми Кейлер')

In [ ]:
#export
def rm_quot_marks(text: str) -> str:
    """Remove quotation marks from `text`."""
    # \u0022\u0027\u00ab\u00bb\u2018\u2019\u201a\u201b\u201c\u201d\u201e\u201f\u2039\u203a\u276e\u276f\u275b\u275c\u275d\u275e\u275f\u2760\u2e42\u301d\u301e\u301f
    return re.sub(r'["\'«»‘’‚‛“”„‟‹›❮❯❛❜❝❞❟❠]','',text)   

In [ ]:
test_eq(rm_quot_marks('"\'«»‘’‚‛“”„‟‹›❮❯❛❜❝❞❟❠'),'')

### Test Text Strings Equality

In [ ]:
#export
def texts_equal(text1: str, text2: str, ignore_e: bool = True, verbose = False)\
        -> Tuple[bool, str, str]:
    """Check if `text1` equals `text2`. Optionally ignore diff between `е` and `ё`."""

    is_equal = 1
    text1, text2 = text1.replace('-',' ').strip(), text2.replace('-',' ').strip()
    if len(text1) != len(text2):
        if verbose: print("Not equal length")
        return False, text1, text2

    words1 = [_.text for _ in list(tokenize(text1))]
    words2 = [_.text for _ in list(tokenize(text2))]
    wc1, wc2 = len(words1), len(words2)
    if wc1 != wc2:
        if verbose: print(f"Not equal words count: {wc1} != {wc2}")
        return False, text1, text2   
    
    text1, text2 = "", ""
    # Per word comparison, assuming wc1 == wc2
    for i in range(len(words1)):
        letters1 = [char for char in words1[i]]
        letters2 = [char for char in words2[i]]        
        if words1[i] != words2[i]:
            is_equal -= 1
            for j in range(min(len(letters1), len(letters2))):
                if letters1[j] == letters2[j]:
                    continue
                else:
                    if ignore_e and letters1[j] in ['е', 'ё'] and letters2[j] in ['е', 'ё']:
                        if verbose: print('е != ё -- норм')
                        is_equal += 1
                    elif letters1[j] in ['-', ' '] and letters2[j] in ['-', ' ']:
                        is_equal += 1
                    else:
                        letters1[j] = letters1[j].upper()
                        letters2[j] = letters2[j].upper()
                        is_equal -= 1
        words1[i], words2[i] = ''.join(letters1), ''.join(letters2)
        text1 = text1 + " " + words1[i]
        text2 = text2 + " " +  words2[i]
        
    return is_equal == 1, text1[1:], text2[1:]

In [ ]:
texts_equal("что-ли а", "что-то и", verbose=False)

(False, 'что ЛИ А', 'что ТО И')

In [ ]:
texts_equal("что-ли а", "что-то и", verbose=True)

(False, 'что ЛИ А', 'что ТО И')

In [ ]:
test_eq(texts_equal("1234", "12345", verbose = False), (False, "1234", "12345"))
#test_stdout(lambda: test_eq(texts_equal("1234", "12345", verbose = True), False), "Not equal length")

test_eq(texts_equal("все", "всё", ignore_e = True, verbose = False), (True, "все", "всё"))
test_eq(texts_equal("все", "всё", ignore_e = False, verbose = False), (False, "всЕ", "всЁ"))
#test_stdout(lambda: texts_equal("все", "всё", ignore_e = False, verbose = True), "всЕ != всЁ")

test_eq(texts_equal("слово ещё одно", "слово ещё одно"), (True,"слово ещё одно", "слово ещё одно"))

In [ ]:
#hide
# test_stdout(lambda: texts_equal("слово ещё одно", "слово ещё одна"), 
#             "однО != однА")

# test_stdout(lambda: texts_equal("слово ещё одно", "слово ещё одно лишнее"), 
#             "Not equal length\nNot equal words count: 3 != 4")

## Pipelines

In [ ]:
#export
def basic_cleaner(text: str) -> str:
    "Basic pipeline: lowercase and collapse whitespaces."
    text = lowercase(text)
    text = collapse_whitespace(text)
    return text

In [ ]:
test_eq(basic_cleaner(
    'Привет   Джон, т.е. Иван, т.к. русский. И т.д.   и т.п.'),
    'привет джон, т.е. иван, т.к. русский. и т.д. и т.п.')

In [ ]:
#export
def russian_cleaner(text, purge_digits=True, _purge_dots=False):
    "Pipeline for cleaning Russian text."

    text = expand_abbreviations(text)
    text = remove_specials(text, purge_digits=purge_digits)
    text = purge_dots(text, purgedots=_purge_dots)
    text = unify_dash_hyphen(text)
    text = rm_quot_marks(text)
    text = collapse_whitespace(text)
    return text

In [ ]:
#export
def russian_cleaner2(text, purge_digits=True, _purge_dots=False):
    "Pipeline for cleaning and lowercase Russian text."

    return russian_cleaner(lowercase(text), purge_digits, _purge_dots)

In [ ]:
test_eq(russian_cleaner(
        'Привет «Джон», т.е.     Иван, т.к. русский... И т.д. и т.п. Ой!. Ага?. / "Стоп"..'),
        'Привет Джон, то есть Иван, так как русский. и так далее. и тому подобное. Ой! Ага? Стоп')

## Sentences Tokenizer

In [ ]:
from razdel import sentenize

In [ ]:
fname = '/home/condor/git/cyrillica/b-ish.txt'
with open(fname) as f:
    text = f.read()

In [ ]:
text

'\ufeff\n\n\n\n\n\n\n\nВсе думают, что знают историю Америки, но когда заходит речь о морских приключениях, мы пересказываем сказки. Взять хотя бы морских чудовищ – пиратов. Вы знали, что они сражались совсем не так, как нам показывают в кино? И что Чёрная Борода боялся, как бы враги не раскрыли его секрет? И что за фокусы с исчезновением показывает Бермудский треугольник? (00:25) Инопланетяне не угоняли звено «девятнадцать», виновник жил у нас дома. Суши вёсла, парень, я – Джейми Кэйлер. Пора порубить мифы на мелкие кусочки… и выкопать сундук… с правдой. \n\n\nАМЕРИКА:  ФАКТЫ  И  ДОМЫСЛЫ\n\n\nЗолотой век пиратства пришёлся на период с тысяча шестьсот  девяностого по тысяча семьсот тридцатый год. Но большинство пиратов быстро отходили от дел. Эдвард Тич.. знаменитый Чёрная Борода.. наводил страх на моря меньше двух лет. Как и многие пираты, он получил раннюю пенсию.. вместе с ударом абордажной сабли. Короткая карьера, зато наследие на века. Но всё, что мы знаем, омыто океаном фантазии.

In [ ]:
text = russian_cleaner(text)
text

'Все думают, что знают историю Америки, но когда заходит речь о морских приключениях, мы пересказываем сказки. Взять хотя бы морских чудовищ – пиратов. Вы знали, что они сражались совсем не так, как нам показывают в кино? И что Чёрная Борода боялся, как бы враги не раскрыли его секрет? И что за фокусы с исчезновением показывает Бермудский треугольник? Инопланетяне не угоняли звено девятнадцать, виновник жил у нас дома. Суши вёсла, парень, я – Джейми Кэйлер. Пора порубить мифы на мелкие кусочки. и выкопать сундук. с правдой. АМЕРИКА: ФАКТЫ И ДОМЫСЛЫ Золотой век пиратства пришёлся на период с тысяча шестьсот девяностого по тысяча семьсот тридцатый год. Но большинство пиратов быстро отходили от дел. Эдвард Тич знаменитый Чёрная Борода наводил страх на моря меньше двух лет. Как и многие пираты, он получил раннюю пенсию вместе с ударом абордажной сабли. Короткая карьера, зато наследие на века. Но всё, что мы знаем, омыто океаном фантазии. Аж поджилки трясутся. Реальность пиратства далеко не

In [ ]:
for s in sentenize(russian_cleaner(text)):
    print(s.text)

Все думают, что знают историю Америки, но когда заходит речь о морских приключениях, мы пересказываем сказки.
Взять хотя бы морских чудовищ – пиратов.
Вы знали, что они сражались совсем не так, как нам показывают в кино?
И что Чёрная Борода боялся, как бы враги не раскрыли его секрет?
И что за фокусы с исчезновением показывает Бермудский треугольник?
Инопланетяне не угоняли звено девятнадцать, виновник жил у нас дома.
Суши вёсла, парень, я – Джейми Кэйлер.
Пора порубить мифы на мелкие кусочки. и выкопать сундук. с правдой.
АМЕРИКА: ФАКТЫ И ДОМЫСЛЫ Золотой век пиратства пришёлся на период с тысяча шестьсот девяностого по тысяча семьсот тридцатый год.
Но большинство пиратов быстро отходили от дел.
Эдвард Тич знаменитый Чёрная Борода наводил страх на моря меньше двух лет.
Как и многие пираты, он получил раннюю пенсию вместе с ударом абордажной сабли.
Короткая карьера, зато наследие на века.
Но всё, что мы знаем, омыто океаном фантазии.
Аж поджилки трясутся.
Реальность пиратства далеко не 

### Set of characters in the origial text

In [ ]:
print(f'Char\tDec\tHex\tPrintable?')
for i,c in enumerate(sorted(set(text))):
    print(f'{c}\t{ord(c)}\t{hex(ord(c))}\t{c.isprintable()}')

Char	Dec	Hex	Printable?
 	32	0x20	True
,	44	0x2c	True
-	45	0x2d	True
.	46	0x2e	True
:	58	0x3a	True
?	63	0x3f	True
А	1040	0x410	True
Б	1041	0x411	True
В	1042	0x412	True
Г	1043	0x413	True
Д	1044	0x414	True
Е	1045	0x415	True
З	1047	0x417	True
И	1048	0x418	True
К	1050	0x41a	True
Л	1051	0x41b	True
М	1052	0x41c	True
Н	1053	0x41d	True
О	1054	0x41e	True
П	1055	0x41f	True
Р	1056	0x420	True
С	1057	0x421	True
Т	1058	0x422	True
У	1059	0x423	True
Ф	1060	0x424	True
Ч	1063	0x427	True
Ы	1067	0x42b	True
Э	1069	0x42d	True
а	1072	0x430	True
б	1073	0x431	True
в	1074	0x432	True
г	1075	0x433	True
д	1076	0x434	True
е	1077	0x435	True
ж	1078	0x436	True
з	1079	0x437	True
и	1080	0x438	True
й	1081	0x439	True
к	1082	0x43a	True
л	1083	0x43b	True
м	1084	0x43c	True
н	1085	0x43d	True
о	1086	0x43e	True
п	1087	0x43f	True
р	1088	0x440	True
с	1089	0x441	True
т	1090	0x442	True
у	1091	0x443	True
ф	1092	0x444	True
х	1093	0x445	True
ц	1094	0x446	True
ч	1095	0x447	True
ш	1096	0x448	True
щ	1097	0x449	True
ы	1099	0x44b	True
ь	11


### Set of characters in the cleaned text

In [ ]:
print(f'Char\tDec\tHex\tPrintable?')
for i,c in enumerate(sorted(set(russian_cleaner2(text)))):
    print(f'{c}\t{ord(c)}\t{hex(ord(c))}\t{c.isprintable()}')

Char	Dec	Hex	Printable?
 	32	0x20	True
,	44	0x2c	True
-	45	0x2d	True
.	46	0x2e	True
:	58	0x3a	True
?	63	0x3f	True
а	1072	0x430	True
б	1073	0x431	True
в	1074	0x432	True
г	1075	0x433	True
д	1076	0x434	True
е	1077	0x435	True
ж	1078	0x436	True
з	1079	0x437	True
и	1080	0x438	True
й	1081	0x439	True
к	1082	0x43a	True
л	1083	0x43b	True
м	1084	0x43c	True
н	1085	0x43d	True
о	1086	0x43e	True
п	1087	0x43f	True
р	1088	0x440	True
с	1089	0x441	True
т	1090	0x442	True
у	1091	0x443	True
ф	1092	0x444	True
х	1093	0x445	True
ц	1094	0x446	True
ч	1095	0x447	True
ш	1096	0x448	True
щ	1097	0x449	True
ы	1099	0x44b	True
ь	1100	0x44c	True
э	1101	0x44d	True
ю	1102	0x44e	True
я	1103	0x44f	True
ё	1105	0x451	True
–	8211	0x2013	True


### Set of the removed/replaced characters

In [ ]:
print(f'Char\tDec\tHex\tPrintable?')
for i,c in enumerate(sorted( set(text).difference(set(russian_cleaner2(text))))):
    print(f'{c}\t{ord(c)}\t{hex(ord(c))}\t{c.isprintable()}')

Char	Dec	Hex	Printable?
А	1040	0x410	True
Б	1041	0x411	True
В	1042	0x412	True
Г	1043	0x413	True
Д	1044	0x414	True
Е	1045	0x415	True
З	1047	0x417	True
И	1048	0x418	True
К	1050	0x41a	True
Л	1051	0x41b	True
М	1052	0x41c	True
Н	1053	0x41d	True
О	1054	0x41e	True
П	1055	0x41f	True
Р	1056	0x420	True
С	1057	0x421	True
Т	1058	0x422	True
У	1059	0x423	True
Ф	1060	0x424	True
Ч	1063	0x427	True
Ы	1067	0x42b	True
Э	1069	0x42d	True


In [ ]:
assert check_no_numbers(russian_cleaner(text))== []

In [ ]:
text = '''Восклицательное предложение! А это какое? Инициалы -- не повод разрывать. Правда, А.С. Пушкин? -- Разумеется, голубчик. (Скобки оставляем.)'''

In [ ]:
for _ in sentenize(text):
    print(_.text)

Восклицательное предложение!
А это какое?
Инициалы -- не повод разрывать.
Правда, А.С. Пушкин?
-- Разумеется, голубчик.
(Скобки оставляем.)


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_text_norm.ipynb.
Converted 02_data.ipynb.
Converted 03_transcribe.ipynb.
Converted index.ipynb.
